In [ ]:
import pandas as pd
import numpy as np
import dash
import flask
import math
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import dash_table
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__)
app.scripts.config.serve_locally = True

app.layout=html.Div(children=[
            html.H1(id='header',children=['Linear Regression Demonstration'],style={
                              'textAlign': 'center',
                              'color': 'blue',
                              'fontSize':50}),
            html.Div(id='instructions',
                     children=['Enter coordinates in table below and watch as Linear Regression model find the line of best fit',
                                                html.Br(),'You can select different metrics to see how the model performs'],
                    style={'textAlign': 'center',
                              'color': 'skyblue',
                              'fontSize':20}              
                    ),
            dcc.Graph(id='adding-rows-graph'),
            dcc.RadioItems(id='residuals',options=[{'label':'Residuals Off','value':'no'},
                                                   {'label':'Residuals On','value':'yes'}],value='no'),
            dcc.RadioItems(id='mean',options=[{'label':'Mean Off','value':'no'},
                                              {'label':'Mean On','value':'yes'}],value='no'),
            html.Br(),
            html.Div(id='metrics'),
            html.Br(),
            html.Div(id='input',children=[
                
            
                dash_table.DataTable(id='table', columns=[{'name':'x','id':'x','type':'numeric'},
                                                      {'name':'y','id':'y','type':'numeric'}],
                                             data=[{'x':5,'y':6}],
                                             editable=True,
                                             row_deletable=True),
                html.Button('Add Row', id='editing-rows-button', n_clicks=0)
                    ])
            
])   

@app.callback(
    Output('table', 'data'),
    [Input('editing-rows-button', 'n_clicks')],
    [State('table', 'data'),
     State('table', 'columns')])
def add_row(n_clicks, rows, columns):
    if n_clicks > 0:
        rows.append({c['id']: '' for c in columns})
    return rows 
@app.callback(
    [Output('adding-rows-graph', 'figure'),
     Output('metrics','children')],
    [Input('table', 'data'),
     Input('table', 'columns'),
     Input('residuals','value'),
     Input('mean','value')])
def display_output(rows, columns,resid,mean):
    df = pd.DataFrame(rows, columns=[c['name'] for c in columns])
    df['rd']=df.x.apply(lambda x: int(round(float(x),0)))
    x=[i for i in range(df.rd.max()+2)]
    mod=LinearRegression().fit(df[['x']],df[['y']])
    df['pred']=mod.predict(df[['x']])
    r2=mod.score(df[['x']],df[['y']])
    mae=mean_absolute_error(df[['y']],df[['pred']])
    rmse=mean_squared_error(df[['y']],df[['pred']])**0.5
    if mean=='yes':
        mn=df.y.mean()
    else:
        mn=[]
    children=['R^2 Score: ' + str(r2),html.Br(),'Mean Absolute Error: ' +str(mae),html.Br(),'Root Mean Squared Error: '+str(rmse)]
    fig=go.Figure()
    fig.add_trace(go.Scatter(
        x=[i for i in df['x']],
        y=[j for j in df['y']],
        mode='markers',
        showlegend=False
    )),
    fig.add_trace(go.Scatter(
        name='Regression Line',
        x=[i for i in x],
        y=[mod.intercept_[0]+mod.coef_[0][0]*x[i] for i in x],
       
    )),
    fig.add_trace(go.Scatter(
        name='Mean Line',
        x=[i for i in x],
        y=[mn for i in range(df.rd.max()+2)],
       
    )),
    if resid=='yes':
        for index in range(len(df.x)):
            if df.y[index]>df.pred[index]:
                fig.add_shape(
                    dict(type='line',x0=df.x[index],y0=df.pred[index],x1=df.x[index],y1=df.y[index]))
            else:
                fig.add_shape(
                    dict(type='line',x0=df.x[index],y0=df.y[index],x1=df.x[index],y1=df.pred[index]))   
      
    return fig,children
                                   
                                   
            

application=app.server
if __name__ == '__main__':
    
    application.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/May/2022 14:49:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 14:49:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 14:49:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 14:49:10] "POST /_dash-update-component HTTP/1.1" 200 -
C:\Users\Alastair Tyson\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

127.0.0.1 - - [24/May/2022 14:49:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 14:49:12] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-pa

127.0.0.1 - - [24/May/2022 14:49:12] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-pa

...

127.0.0.1 - - [24/May/2022 14:50:24] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [24/May/2022 14:50:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 14:50:28] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-pa

127.0.0.1 - - [24/May/2022 14:50:28] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Alastair Tyson\anaconda3\lib\site-pa

127.0.0.1 - - [24/May/2022 14:50:30] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [24/May/2022 14:50:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 14:53:50] "POST /_dash-update-component HTTP/1.1" 200 -
